# **Proyecto Final Primera Entrega**

## **Introducción**
El análisis de datos en el mercado inmobiliario de alquiler es fundamental en la industria de bienes raíces, ya que influye en las decisiones de arrendadores, inquilinos, inversores y autoridades urbanas.

El conjunto de datos “Apartment for Rent Classified” proviene del repositorio UCI Machine Learning y contiene anuncios clasificados de apartamentos en alquiler en Estados Unidos, con atributos como precio, área (pies cuadrados), amenidades (p. ej., internet, piscina) y ubicación geográfica.

Consta de 10.000 registros y 22 columnas, admitiendo tareas de clasificación, regresión y agrupamiento

Estas son las variables:

| Campo          | Descripción                                                                                                        |
| -------------- | ------------------------------------------------------------------------------------------------------------------ |
| d              | identificador único del apartamento                                                                                |
| category       | categoría del anuncio                                                                                              |
| title          | título del apartamento                                                                                             |
| body           | descripción del apartamento                                                                                        |
| amenities      | comodidades (ej. aire acondicionado, cancha de básquetbol, cable, gimnasio, internet, alberca, refrigerador, etc.) |
| bathrooms      | número de baños                                                                                                    |
| bedrooms       | número de recámaras                                                                                                |
| currency       | moneda del precio                                                                                                  |
| fee            | cuota / comisión                                                                                                   |
| has\_photo     | indica si el apartamento tiene foto                                                                                |
| pets\_allowed  | qué mascotas están permitidas (perros, gatos, etc.)                                                                |
| price          | precio de renta del apartamento                                                                                    |
| price\_display | precio convertido en un formato para mostrar al lector                                                             |
| price\_type    | precio en USD                                                                                                      |
| square\_feet   | tamaño del apartamento (en pies cuadrados)                                                                         |
| address        | dirección del apartamento                                                                                          |
| cityname       | ciudad donde se ubica el apartamento                                                                               |
| state          | estado donde se ubica el apartamento                                                                               |
| latitude       | latitud donde se ubica el apartamento                                                                              |
| longitude      | longitud donde se ubica el apartamento                                                                             |
| source         | origen del anuncio                                                                                                 |
| time           | fecha de creación del anuncio                                                                                      |


## **Antecedentes**


## **Planteamiento del problema**

## **EDA**

## **Data Wrangling**

# Conclusiones

# Referencias